# Assignment 4
## Understaning scaling of linear algebra operations on Apache Spark using Apache SystemML

In this assignment we want you to understand how to scale linear algebra operations from a single machine to multiple machines, memory and CPU cores using Apache SystemML. Therefore we want you to understand how to migrate from a numpy program to a SystemML DML program. Don't worry. We will give you a lot of hints. Finally, you won't need this knowledge anyways if you are sticking to Keras only, but once you go beyond that point you'll be happy to see what's going on behind the scenes. As usual, we run some import statements:

In [1]:
!pip install --upgrade systemml

Requirement already up-to-date: systemml in /gpfs/global_fs01/sym_shared/YPProdSpark/user/s6f3-591775d08ea93b-f79756be55b6/.local/lib/python3.5/site-packages (1.2.0)
Requirement not upgraded as not directly required: numpy>=1.8.2 in /usr/local/src/conda3_runtime.v47/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from systemml) (1.13.3)
Requirement not upgraded as not directly required: Pillow>=2.0.0 in /usr/local/src/conda3_runtime.v47/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from systemml) (4.2.1)
Requirement not upgraded as not directly required: scipy>=0.15.1 in /usr/local/src/conda3_runtime.v47/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from systemml) (1.0.0)
Requirement not upgraded as not directly required: scikit-learn in /usr/local/src/conda3_runtime.v47/home/envs/DSX-Python35-Spark/lib/python3.5/site-packages (from systemml) (0.19.1)
Requirement not upgraded as not directly required: pandas in /usr/local/src/conda3_runtime.v47/home/en

In [2]:
from systemml import MLContext, dml
import numpy as np
import time

Then we create an MLContext to interface with Apache SystemML. Note that we pass a SparkSession object as parameter so SystemML now knows how to talk to the Apache Spark cluster

In [3]:
ml = MLContext(spark)

Now we create some large random matrices to have numpy and SystemML crunch on it

In [4]:
u = np.random.rand(1000,10000)
s = np.random.rand(10000,1000)
w = np.random.rand(1000,1000)

Now we implement a short one-liner to define a very simple linear algebra operation

In case you are not familiar with matrix-matrix multiplication: https://en.wikipedia.org/wiki/Matrix_multiplication

sum(U' * (W . (U * S)))


| Legend        |            |   
| ------------- |-------------| 
| '      | transpose of a matrix | 
| * | matrix-matrix multiplication      |  
| . | scalar multiplication      |   



In [5]:
start = time.time()
res = np.sum(u.T.dot(w * u.dot(s)))
print (time.time()-start)

0.6783435344696045


As you can see this executes perfectly fine. Note that this is even a very efficient execution because numpy uses a C/C++ backend which is known for it's performance. But what happens if U, S or W get such big that the available main memory cannot cope with it? Let's give it a try:

In [6]:
u = np.random.rand(10000,100000)
s = np.random.rand(100000,10000)
w = np.random.rand(10000,10000)

After a short while you should see a memory error. This is because the operating system process was not able to allocate enough memory for storing the numpy array on the heap. Now it's time to re-implement the very same operations as DML in SystemML, and this is your task. Just replace all ###your_code_goes_here sections with proper code, please consider the following table which contains all DML syntax you need:

| Syntax        |            |   
| ------------- |-------------| 
| t(M)      | transpose of a matrix, where M is the matrix | 
| %*% | matrix-matrix multiplication      |  
| * | scalar multiplication      |   

## Task

In [7]:
script = """
res = sum(t(U) %*% (W * (U %*% S)))
"""

To get consistent results we switch from a random matrix initialization to something deterministic

In [8]:
u = np.arange(100000).reshape((100, 1000))
s = np.arange(100000).reshape((1000, 100))
w = np.arange(10000).reshape((100, 100))

In [9]:
prog = dml(script).input('U', u).input('S', s).input('W', w).output('res')
res = ml.execute(prog).get('res')
print (res)

SystemML Statistics:
Total execution time:		0.074 sec.
Number of executed Spark inst:	3.


1.2526052592176266e+28


If everything runs fine you should get *1.25260525922e+28* as result. Feel free to submit your DML script to the grader now!

### Submission

In [10]:
!rm -f rklib.py
!wget https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py

--2018-11-20 08:31:04--  https://raw.githubusercontent.com/romeokienzler/developerWorks/master/coursera/ai/rklib.py
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.48.133
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.48.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 2289 (2.2K) [text/plain]
Saving to: ‘rklib.py’

100%[======================================>] 2,289       --.-K/s   in 0s      

2018-11-20 08:31:04 (17.3 MB/s) - ‘rklib.py’ saved [2289/2289]



In [11]:
from rklib import submit
key = "esRk7vn-Eeej-BLTuYzd0g"
part = "fUxc8"

email = "duy.nguyen2015@ict.jvn.edu.vn"
secret = "YgE1lKNbU0eWEz36"
submit(email, secret, key, part, [part], script)

Submission successful, please check on the coursera grader page for the status
-------------------------
{"elements":[{"itemId":"P1p3F","id":"tE4j0qhMEeecqgpT6QjMdA~P1p3F~BHho5ezREeiv3QplIMwUHA","courseId":"tE4j0qhMEeecqgpT6QjMdA"}],"paging":{},"linked":{}}
-------------------------
